<a href="https://colab.research.google.com/github/zhuoyingcai/NYC-Housing-Prices-Analysis/blob/master/NYC_Housing_Prices_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Section

In [0]:
import os

# Data
import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt


# Path Functions

In [0]:
def ensure_dir(path_to_dir):
  try:
      os.makedirs(path_to_dir)
  except FileExistsError:
      # directory already exists
      pass

# Config

In [243]:
# path config
annualizedSalesUpdate_data_dir = '/content/drive/My Drive/Colab Notebooks/dataScience/annualizedSalesUpdate'
data_dir = '/content/drive/My Drive/Colab Notebooks/dataScience/data'

# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# make sure dir exist
ensure_dir(data_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load and summarize data

In [0]:
df = pd.DataFrame()
boroughs = ['manhattan','bronx','brooklyn','queens','statenisland']

for year in range(2006,2011):
  for borough in boroughs:
    path = annualizedSalesUpdate_data_dir + '/%s_%s.xls' % (year, borough)
    data = pd.read_excel(path, header=3)
    df = df.append(data, sort=False)

In [0]:
# for year in range(2011,2019):
#   for borough in boroughs:
#     path = annualizedSalesUpdate_data_dir + '/%s_%s.xls' % (year, borough)
#     data = pd.read_excel(path, header=4)
#     df = df.append(data, sort=False)


In [276]:
# df.head(20)
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 457659 entries, 0 to 6137
Data columns (total 21 columns):
BOROUGH                           457659 non-null int64
NEIGHBORHOOD                      457659 non-null object
BUILDING CLASS CATEGORY           457659 non-null object
TAX CLASS AT PRESENT              457659 non-null object
BLOCK                             457659 non-null int64
LOT                               457659 non-null int64
EASE-MENT                         457659 non-null object
BUILDING CLASS AT PRESENT         457659 non-null object
ADDRESS                           457659 non-null object
APARTMENT NUMBER                  457659 non-null object
ZIP CODE                          457659 non-null int64
RESIDENTIAL UNITS                 457659 non-null int64
COMMERCIAL UNITS                  457659 non-null int64
TOTAL UNITS                       457659 non-null int64
LAND SQUARE FEET                  457659 non-null int64
GROSS SQUARE FEET                 457659 non